# Plot histograms and weights

Plot histograms of r-band magnitudes for CFIS in W3 field:
- unmatched catalogue
- matched with DEEP2+3
- weights = ratio of matched to unmatched density

In [ ]:
import os
import sys

from functions import *
import params_ps3pi_cfis as params

In [ ]:
path = f"{os.environ['CONDA_PREFIX']}/../shapepipe/lib/python3.8/site-packages/shapepipe-0.0.2-py3.8.egg"
sys.path.append(path)
from shapepipe.utilities.canfar import download

## Set up paths and parameters

In [ ]:
path = os.getcwd() + '/'
bands = params.bands
output_path = params.output_path
output_name = params.output_name
temp_path = params.temp_path

spectral_path = params.spectral_path
spectral_names = params.spectral_names
spectral_surveys = params.spectral_surveys

cat = MakeCatalogs('ps3pi_cfis', bands, temp_path, output_name, output_path)

matched_cat_name = 'CFIS_matched_deep_2_3_catalog_R_preprocessed.csv'
unmatched_cat_name = 'MediumDeep_IZG_CFHT_U_CFIS_R_catalog_unmatched.csv'

cat_dir = './catalogs/'

## Get input catalogue(s)

In [ ]:
vos_dir = 'vos:cfis/cosmostat/catalogues/multi_band_matched/W3_deep23'
for name in [matched_cat_name, unmatched_cat_name]:
    download(f'{vos_dir}/{name}', f'{cat_dir}/{name}', verbose=True)

In [ ]:
df_matched = pd.read_csv(f'{cat_dir}/{matched_cat_name}')
df_matched = df_matched.dropna()

In [ ]:
df_unmatched = pd.read_csv(f'{cat_dir}/{unmatched_cat_name}')

## Weights

Account for the difference in numbers between matched and unmatched catalogue, by introducing weights.
Here, this difference is only considered in the $r$-band magnitude number density. The weight is the ratio
of objects between the full (unmatched) and matched catalogues.

### Compute weights`

In [ ]:
# Number of bins
n = 100

# band
column = 'r'

# density (histogram) of full (unmatched) catalogue
udensity, ubins_edge = np.histogram(df_unmatched[column].values, bins=n, density=True)
ubin_min = ubins_edge[0]
ubin_max = ubins_edge[-1]
ustep = (ubin_max - ubin_min)/n

# density (histogram) of matched catalogue
density, bins_edge = np.histogram(df_matched[column].values, bins=n, density=True)
bin_min = bins_edge[0]
bin_max = bins_edge[-1]
step = (bin_max - bin_min)/n

ID = np.arange(0, len(df_matched))
df_matched.insert(loc=0, value=ID, column='ID')
df_matched.sort_values(by=column, inplace=True)

weights, uweights = [], []
n_min = 0
n_umin = 0
h1, h2 = [], []
for i in range(len(df_matched)):
    for k in range(n_min, n):
        if bin_min+step*(k+1) < df_matched[column].values[i]:
            continue
        else:
            weights.append(density[k])
            n_min = k
            h1.append(df_matched[column].values[i])
            break 
        
    for k in range(n_umin, n):
        if ubin_min+ustep*(k+1) < df_matched[column].values[i]:
            continue
        else:
            uweights.append(udensity[k])
            n_umin = k
            h2.append(df_matched[column].values[i])
        
        break 
  
# Final weight = ratio of densities
w = np.array(uweights) / np.array(weights)

### Plot number densities and corresponding weight

In [ ]:
fontsize = 18
linewidth = 2

fig = plt.figure(figsize=(8, 6), tight_layout=False)

ax1 = fig.add_subplot(111)
ax2 = ax1.twinx()
ax3 = ax2.twiny()
ax1.get_shared_x_axes().join(ax2, ax3)
ax2.set_ylabel('normalized frequency', fontsize=fontsize)

ax1.set_facecolor('white')
ax1.set_xlabel(r'$r$', fontsize=fontsize)
ax1.set_ylabel('weight', fontsize=fontsize)

ax1.plot(df_matched[column].to_numpy(), w, label='weight', lw=linewidth, color='r')
ax1.axhline(1, color='k', linestyle='--')

ax2.plot(h1, np.array(weights), linestyle='dotted', linewidth=linewidth, color='grey',
         alpha=0.8, label='matched')
ax3.fill_between(h1, np.array(weights), color='grey', alpha=0.5)

ax2.plot(h2, np.array(uweights), linestyle='dashed', linewidth=linewidth, color='k',
         alpha=0.8, label='full')
ax3.fill_between(h2, np.array(uweights), color='grey', alpha=0.5)

# Remove ticks on second x-axis
plt.xticks([])

legend = ax2.legend(loc='upper left', shadow=True, fontsize='x-large')
legend = ax1.legend(loc=(0.02,0.7), shadow=True, fontsize='x-large')

# order
ax3.zorder = 1 # fills in back
ax1.zorder = 2 # then the line
ax2.zorder = 3 # then the points
ax1.patch.set_visible(False)

plt.savefig('mag_weight_dist_deep23.pdf', bbox_inches='tight', transparent=True)
plt.show()